**The issue with just depleting the p-value or deflating on trials is that they might be realtted and not independent and for that reason, you might be deflating them at too extreme a rate**

So you have walk-forward back-tests, which is the back tests you and I know. And then quite naturally, we have backtests on simulated data. 

I can imagine another type of backtests and that is superimposing signals from prices onto new prices to recreate new prices based on multiple scenarios. 

Another potential is to have multiple time-series and then to use back-testing as a tool to fit on - it is going to be harder to fit on multiple ones, in the sense that you do cross-validation. 

Naturally walk forward leads to higher variance, higher variance leads to more false discovery.



https://github.com/jjakimoto/finance_ml/blob/master/examples/miscellaneous/Ch 3.ipynb

From here forward you would like to learn more about Chapter 3

In [56]:
import pandas as pd

df = pd.read_csv("datasets/Google.csv")
df.index = pd.DatetimeIndex(df['Date'].values)
close = df["Close"]

In [57]:
import numpy as np
import pandas as pd

### This really gets a moving average of the daily volatility. 
def get_daily_vol(close, span=100):
    # Easy, search the index for where the next index is a day away from previous index
    # It will return duplicate indexes, because its relative to before and after
    # Returns closest index to another. 
    # It is a hack - where index should be added to maintain order.
    
    # The insertion index is the place to add your value to keep the index in sorted order
    use_idx = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    use_idx = use_idx[use_idx > 0]
    # Get rid of duplications in index
    use_idx = np.unique(use_idx)
    prev_idx = pd.Series(close.index[use_idx - 1], index=close.index[use_idx])
    ret = close.loc[prev_idx.index] / close.loc[prev_idx.values].values - 1
    # Ewm is an exponentially weighted moving average, various parameters can be tweaked 
    vol = ret.ewm(span=span).std()
    return vol

In [66]:
### Time to simplify the formula, to write it in as little code as possible.
### This value is slightly better but the formula is better.
### I have noticed, this is the gaps, he doesn't take the returns of the
### gaps, that is fine with me, I might still wonder about the
## justification of that, but it is done. 
## basically you can ask him why he is leaving out the gap. 
#### Interesting, my EW takes friday out.
#### His takes monday out

import numpy as np
import pandas as pd

### This really gets a moving average of the daily volatility. 
def daily_moving_vol(close, span=100):
    if close.shape[0]==close.index.drop_duplicates().shape[0]:
        # Ewm is an exponentially weighted moving average, various parameters can be tweaked like span
        vol = ((close/close.shift(-1)-1).drop_duplicates().dropna()).ewm(span=span).std()
        return vol
    else:
        print("duplicate indexes - fix")



In [67]:
def get_daily_vol_sans_monday(close, span=100):
    days = pd.Series(close[1:].index.append(close.index.shift(1,freq="D")[-1:]) - close.index  < pd.Timedelta(days=2))
    days = days.shift(1).dropna()
    days = days[days==True]

    ret = ((close/close.shift(-1)-1).drop_duplicates().dropna())

    ew = ret.iloc[days.index].ewm(span=span).std()
    return ew

In [64]:
vol1 = get_daily_vol(close)
vol1 = vol1.to_frame()
vol1["date"] = list(vol1.index)
vol1["dow"] = vol1["date"].dt.day_name()
vol1.head(10)

,Close,date,dow
2004-08-23,NaN,2004-08-23,Monday
2004-08-24,0.036396,2004-08-24,Tuesday
2004-08-25,0.029930,2004-08-25,Wednesday
2004-08-26,0.027366,2004-08-26,Thursday
2004-08-30,0.029365,2004-08-30,Monday
2004-08-31,0.026605,2004-08-31,Tuesday
2004-09-01,0.024794,2004-09-01,Wednesday
2004-09-02,0.024097,2004-09-02,Thursday
2004-09-07,0.023610,2004-09-07,Tuesday
2004-09-08,0.022400,2004-09-08,Wednesday


In [65]:
vol_m = daily_moving_vol(df["Close"])
vol_m = vol_m.to_frame()
vol_m["date"] = list(vol_m.index)
vol_m["dow"] = vol_m["date"].dt.day_name()
vol_m.head(10)

,Close,date,dow
2004-08-19,NaN,2004-08-19,Thursday
2004-08-20,0.045020,2004-08-20,Friday
2004-08-23,0.058456,2004-08-23,Monday
2004-08-24,0.047500,2004-08-24,Tuesday
2004-08-25,0.041005,2004-08-25,Wednesday
2004-08-26,0.038556,2004-08-26,Thursday
2004-08-27,0.039770,2004-08-27,Friday
2004-08-30,0.036647,2004-08-30,Monday
2004-08-31,0.034956,2004-08-31,Tuesday
2004-09-01,0.033110,2004-09-01,Wednesday


In [68]:
vol_d = get_daily_vol_sans_monday(df["Close"])
vol_d = vol_d.to_frame()
vol_d["date"] = list(vol_d.index)
vol_d["dow"] = vol_d["date"].dt.day_name()
vol_d.head(10)

,Close,date,dow
2004-08-20,NaN,2004-08-20,Friday
2004-08-24,0.000493,2004-08-24,Tuesday
2004-08-25,0.004299,2004-08-25,Wednesday
2004-08-26,0.015255,2004-08-26,Thursday
2004-08-27,0.024609,2004-08-27,Friday
2004-08-31,0.022998,2004-08-31,Tuesday
2004-09-01,0.022231,2004-09-01,Wednesday
2004-09-02,0.020847,2004-09-02,Thursday
2004-09-03,0.020728,2004-09-03,Friday
2004-09-08,0.019472,2004-09-08,Wednesday


In [21]:
########## Okay above, the volume has been delt with.

In [ ]:
Isinstance is like is type basically, can include stuff like class too.

>>> isinstance('foo', basestring)
True
>>> isinstance('foo', float)
False


When the CUSUM method is applied to changes in mean, it can be used for step detection of a time series.
In statistical quality control, the CUSUM (or cumulative sum control chart) is a sequential analysis technique developed by E. S. Page of the University of Cambridge.

In [69]:
import numbers
# It is basically a cummulative regime shift of more than 10% after
# which it essentially gets reset.

def cusum_filter(close, h):
    # asssum that E y_t = y_{t-1}
    t_events = []
    s_pos, s_neg = 0, 0
    ret = close.pct_change().dropna()
    diff = ret.diff().dropna()
    # time variant threshold
    if isinstance(h, numbers.Number):
        h = pd.Series(h, index=diff.index)
    h = h.reindex(diff.index, method='bfill')
    h = h.dropna()
    for t in h.index:
        s_pos = max(0, s_pos + diff.loc[t])
        s_neg = min(0, s_neg + diff.loc[t])
        if s_pos > h.loc[t]:
            s_pos = 0
            t_events.append(t)
        elif s_neg < -h.loc[t]:
            s_neg = 0
            t_events.append(t)
    return pd.DatetimeIndex(t_events)

In [77]:
vol = get_daily_vol(close)

In [78]:
## Interesting, you don't want the level of volatility to effect the selection
## so instead use it is a threshold of selection.

sampled_idx = cusum_filter(close, vol)
sampled_idx
sampled_idx.shape

(1513,)

In [82]:
## This is simply getting time + 1 
def get_t1(close, t_events, num_days):
    t1 = close.index.searchsorted(t_events + pd.Timedelta(days=num_days))
    t1 = t1[t1 < close.shape[0]]
    t1 = pd.Series(close.index[t1], index=t_events[:t1.shape[0]])
    return t1

In [83]:
t1 = get_t1(close, sampled_idx, num_days=1)
print(t1.shape)
t1.head()

(1513,)


2004-08-24   2004-08-25
2004-08-25   2004-08-26
2004-08-27   2004-08-30
2004-08-31   2004-09-01
2004-09-02   2004-09-03
dtype: datetime64[ns]

In [84]:
## This simply implements the three barrier method. 

def apply_ptslt1(close, events, ptsl, molecule):
    """Return datafram about if price touches the boundary"""
    # Sample a subset with specific indices
    _events = events.loc[molecule]
    # Time limit
    
    out = pd.DataFrame(index=_events.index)
    # Set Profit Taking and Stop Loss
    if ptsl[0] > 0:
        pt = ptsl[0] *  _events["trgt"]
    else:
        # Switch off profit taking
        pt = pd.Series(index=_events.index)
    if ptsl[1] > 0:
        sl = -ptsl[1] * _events["trgt"]
    else:
        # Switch off stop loss
        sl = pd.Series(index=_events.index)
    # Replace undifined value with the last time index
    time_limits = _events["t1"].fillna(close.index[-1])
    for loc, t1 in time_limits.iteritems():
        df = close[loc:t1]
        # Change the direction depending on the side
        df = (df / close[loc] - 1) * _events.at[loc, 'side']
        # print(df)
        # print(loc, t1, df[df < sl[loc]].index.min(), df[df > pt[loc]].index.min())
        out.at[loc, 'sl'] = df[df < sl[loc]].index.min()
        out.at[loc, 'pt'] = df[df > pt[loc]].index.min()
    out['t1'] = _events['t1'].copy(deep=True)
    return out

In [85]:
def get_3barriers(close, t_events, ptsl, trgt, min_ret=0, num_threads=1,
                  t1=False, side=None):
    # Get sampled target values
    trgt = trgt.loc[t_events]
    trgt = trgt[trgt > min_ret]
    # Get time boundary t1
    if t1 is False:
        t1 = pd.Series(pd.NaT, index=t_events)
    # Define the side
    if side is None:
        _side = pd.Series(1., index=trgt.index)
        _ptsl = [ptsl, ptsl]
    else:
        _side = side.loc[trgt.index]
        _ptsl = ptsl[:2]
    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': _side}, axis=1)
    events = events.dropna(subset=['trgt'])
    time_idx = apply_ptslt1(close, events, _ptsl, events.index)
    # Skip when all of barrier are not touched
    events['t1'] = time_idx.dropna(how='all').min(axis=1)
    events = events.drop('side', axis=1)
    return events

In [86]:
trgt = vol
events = get_3barriers(close, t_events=sampled_idx, trgt=vol,
                       ptsl=1, t1=t1)
events.head()

/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


,t1,trgt
2004-08-24,2004-08-25,0.036396
2004-08-25,2004-08-26,0.029930
2004-08-31,2004-09-01,0.026605
2004-09-02,2004-09-03,0.024097
2004-09-07,2004-09-08,0.023610


In [88]:


def get_bins(events, close):
    # Prices algined with events
    events = events.dropna(subset=['t1'])
    px = events.index.union(events['t1'].values).drop_duplicates()
    px = close.reindex(px, method='bfill')
    # Create out object
    out = pd.DataFrame(index=events.index)
    out['ret'] = px.loc[events['t1'].values].values / px.loc[events.index] - 1.
    if 'side' in events:
        out['ret'] *= events['side']
    out['bin'] = np.sign(out['ret'])
    if 'side' in events:
        out.loc[out['ret'] <= 0, 'bin'] = 0
    return out




bins = get_bins(events, close)
bins.head()



,ret,bin
2004-08-24,0.010775,1.0
2004-08-25,0.018019,1.0
2004-08-31,-0.020709,-1.0
2004-09-02,-0.014777,-1.0
2004-09-07,0.007088,1.0


In [91]:
bins["bin"].value_counts()

 1.0    630
-1.0    586
 0.0      1
Name: bin, dtype: int64

In [92]:
def drop_labels(events, min_pct=0.05):
    while True:
        df = events['bin'].value_counts(normalize=True)
        if df.min() > min_pct or df.shape[0] < 3:
            break
        print('dropped label', df.argmin(), df.min())
        events = events[events['bin'] != df.argmin()]
    return events

In [93]:
dropped_bins = drop_labels(bins)
print(bins.shape)
print(dropped_bins.shape)

dropped label 0.0 0.0008216926869350862
(1217, 2)
(1216, 2)


/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


In [94]:
bins = dropped_bins

In [96]:
def get_3barriers(close, t_events, ptsl, trgt, min_ret=0, num_threads=1,
                  t1=False, side=None):
    # Get sampled target values
    trgt = trgt.loc[t_events]
    trgt = trgt[trgt > min_ret]
    # Get time boundary t1
    if t1 is False:
        t1 = pd.Series(pd.NaT, index=t_events)
    # Define the side
    if side is None:
        _side = pd.Series(1., index=trgt.index)
        _ptsl = [ptsl, ptsl]
    else:
        _side = side.loc[trgt.index]
        _ptsl = ptsl[:2]
    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': _side}, axis=1)
    events = events.dropna(subset=['trgt'])
    time_idx = apply_ptslt1(close, events, _ptsl, events.index)
    # Skip when all of barrier are not touched
    time_idx = time_idx.dropna(how='all')
    events['t1_type'] = time_idx.idxmin(axis=1)
    events['t1'] = time_idx.min(axis=1)
    if side is None:
        events = events.drop('side', axis=1)
    return events

def get_bins(events, close):
    # Prices algined with events
    events = events.dropna(subset=['t1'])
    px = events.index.union(events['t1'].values).drop_duplicates()
    px = close.reindex(px, method='bfill')
    # Create out object
    out = pd.DataFrame(index=events.index)
    out['ret'] = px.loc[events['t1'].values].values / px.loc[events.index] - 1.
    if 'side' in events:
        out['ret'] *= events['side']
    out['bin'] = np.sign(out['ret'])
    # 0 when touching vertical line
    out['bin'].loc[events['t1_type'] == 't1'] = 0
    if 'side' in events:
        out.loc[out['ret'] <= 0, 'bin'] = 0
    return out

In [97]:
t1 = get_t1(close, sampled_idx, num_days=1)
events = get_3barriers(close, t_events=sampled_idx, trgt=trgt,
                       ptsl=1, t1=t1)
events.head()

/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


,t1,trgt,t1_type
2004-08-24,2004-08-25,0.036396,t1
2004-08-25,2004-08-26,0.029930,t1
2004-08-31,2004-09-01,0.026605,t1
2004-09-02,2004-09-03,0.024097,t1
2004-09-07,2004-09-08,0.023610,t1


In [99]:
## Easy t1 means you collapsed against the verticle.
## I think this is where you think about overlap
events["t1_type"].value_counts()

t1    906
pt    180
sl    131
Name: t1_type, dtype: int64

In [100]:
bins = get_bins(events, close)
bins.head()

,ret,bin
2004-08-24,0.010775,0.0
2004-08-25,0.018019,0.0
2004-08-31,-0.020709,0.0
2004-09-02,-0.014777,0.0
2004-09-07,0.007088,0.0


In [101]:
## Well it is easy to gauge what bins are doing.
bins["bin"].value_counts()

 0.0    906
 1.0    180
-1.0    131
Name: bin, dtype: int64

In [105]:
import talib
import numpy as np

def macd_side(close):
    macd, signal, hist = talib.MACD(close.values)
    hist = pd.Series(hist).fillna(1).values
    return pd.Series(2 * ((hist > 0).astype(float) - 0.5), index=close.index[-len(hist):])

In [106]:
import numpy as np

vol = get_daily_vol(close)
sampled_idx = cusum_filter(close, vol)
t1 = get_t1(close, sampled_idx, num_days=1)
side =  macd_side(close)
events = get_3barriers(close, t_events=sampled_idx, trgt=vol,
                       ptsl=[1, 2], t1=t1, side=side)
events.head()

/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


,side,t1,trgt,t1_type
2004-08-24,1.0,2004-08-25,0.036396,t1
2004-08-25,1.0,2004-08-26,0.029930,t1
2004-08-31,1.0,2004-09-01,0.026605,t1
2004-09-02,1.0,2004-09-03,0.024097,t1
2004-09-07,1.0,2004-09-08,0.023610,t1


In [107]:
bins = get_bins(events, close)
bins.head()


,ret,bin
2004-08-24,0.010775,0.0
2004-08-25,0.018019,0.0
2004-08-31,-0.020709,0.0
2004-09-02,-0.014777,0.0
2004-09-07,0.007088,0.0


In [110]:
bins["bin"].value_counts()

0.0    1054
1.0     163
Name: bin, dtype: int64

In [112]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
x = np.hstack([events['side'].values[:, np.newaxis], close.loc[events.index].values[:, np.newaxis]])
y = bins['bin'].values
clf.fit(x, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [116]:


clf.predict(x)



array([0., 0., 0., ..., 0., 0., 0.])